In [1]:
import traitlets
import traceback

import numpy as np
import ipywidgets.widgets as widgets

from jetbot import Camera, bgr8_to_jpeg
from jetbot import Robot

from IPython.display import display

robot = Robot()

In [2]:
import tensorflow as tf

physical_devices = tf.config.experimental.list_physical_devices('GPU')
if physical_devices:
  for dev in physical_devices:
    tf.config.experimental.set_memory_growth(dev, True)

In [3]:
#model = tf.saved_model.load("saved_model/depth-v1")
model = tf.saved_model.load("saved_model/depth-v1-trt")
#model = tf.saved_model.load("/home/jetbot/depth-v1-trt")

In [4]:
infer = model.signatures["serving_default"]
inputs = infer.structured_input_signature[1]
print(inputs)

l2state = tf.Variable(tf.ones((1,) + inputs['l2state'].shape[1:], dtype=tf.float32))
l3state = tf.Variable(tf.ones((1,) + inputs['l3state'].shape[1:], dtype=tf.float32))
l4state = tf.Variable(tf.ones((1,) + inputs['l4state'].shape[1:], dtype=tf.float32))

{'l3state': TensorSpec(shape=(None, 7, 7, 160), dtype=tf.float32, name='l3state'), 'l4state': TensorSpec(shape=(None, 7, 7, 32), dtype=tf.float32, name='l4state'), 'odometry': TensorSpec(shape=(None, 6), dtype=tf.float32, name='odometry'), 'input': TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name='input'), 'l2state': TensorSpec(shape=(None, 7, 7, 256), dtype=tf.float32, name='l2state')}


In [5]:
camera = Camera.instance(width=320, height=240, fps=10, capture_width=320, capture_height=240)

cam_img = widgets.Image(format='jpeg', width=224, height=224)
depth_img = widgets.Image(format='jpeg', width=224, height=224)
clearance = widgets.FloatSlider(description='clearance', min=0.0, max=8.0, orientation='vertical')
tz = widgets.FloatSlider(description='tz', min=0.0, max=1.0, orientation='vertical')
rx = widgets.FloatSlider(description='rx', min=-1.0, max=1.0, orientation='vertical')
l2 = widgets.FloatSlider(description='l2', min=0.0, max=10.0, orientation='vertical')

In [6]:
def proc_camera(img):
  w = img.shape[-2]
  h = img.shape[-3]
  if h > w:
    img = tf.image.crop_to_bounding_box(img, (h - w) // 2, 0, w, w)
  elif h < w:
    img = tf.image.crop_to_bounding_box(img, 0, (w - h) // 2, h, h)

  img = tf.image.resize(
    img, (224, 224), antialias=True
  )
    
  return img

In [21]:
camera.unobserve_all()
robot.stop()
#camera.stop()
#camera.start()

avg_weight = 0.75
clearance_min = 1.0

l2state.assign(tf.random.uniform(l2state.shape))
l3state.assign(tf.random.uniform(l3state.shape))
l4state.assign(tf.random.uniform(l4state.shape))

def get_odom():
    tz.value = robot.left_motor.value + robot.right_motor.value
    rx.value = robot.right_motor.value - robot.left_motor.value
    return tf.constant([[rx.value, 0, 0, 0, 0, tz.value]])

def update_motors():
    max_speed = 0.4
    min_speed = -0.2
    #return
    robot.left_motor.value = max_speed
    
    new_right = robot.right_motor.value
    if clearance.value < clearance_min:
        #new_right *= 0.9
        new_right -= 0.01
    else:
        new_right += 0.01
        
    if new_right > max_speed:
        new_right = max_speed
    elif new_right < min_speed:
        new_right = min_speed
        
    robot.right_motor.value = new_right

@tf.function
def cam_update_func(x, odom):
    x = tf.cast(x, tf.float32)
    x = tf.expand_dims(x, 0)
    x /= 255.0
    depth, l2state_, l3state_, l4state_ = model([
        x, odom, l2state, l3state, l4state
    ], training=True)
    l2state.assign(l2state_, read_value=False)
    l3state.assign(l3state_, read_value=False)
    l4state.assign(l4state_, read_value=False)
    depth += 1.0
    depth *= 0.5
    depth_m = depth * 8.0
    depth_m = depth_m[0, :14, :, :]
    depth = tf.broadcast_to(depth, depth.shape[:-1] + (3,))
    depth = tf.clip_by_value(depth, 0, 1)
    depth *= 255
    depth = tf.cast(depth, tf.uint8)
    depth = tf.squeeze(depth, 0)
    return depth, depth_m
    
rep_test = False
def cam_update(change):
    global l2state, l3state, l4state, rep_test
    try:
        x = change['new']
        
        if not rep_test:
            x2 = tf.cast(x, tf.float32)
            rep_test = True
            print(x.shape)
            print(tf.reduce_mean(x2).numpy())
            print(tf.math.reduce_std(x2).numpy())
            
        x = tf.cast(x, tf.uint8)
            
        x = proc_camera(x)
        cam_img.value = bgr8_to_jpeg(x.numpy())
        
        depth, depth_m = cam_update_func(x, get_odom())
        
        depth_img.value = bgr8_to_jpeg(depth.numpy())
        l2.value = tf.reduce_mean(l2state).numpy() * 10
        
        closest_m = tf.math.reduce_mean(depth_m) - tf.math.reduce_std(depth_m)
        clearance.value = clearance.value * (1.0 - avg_weight) + closest_m.numpy() * avg_weight
        
        update_motors()
    except:
        camera.unobserve_all()
        traceback.print_exc()
        
camera.start()
camera.observe(cam_update, names='value')
display(widgets.HBox([cam_img, clearance, rx, tz, l2, depth_img]))

(240, 320, 3)


22.452457
28.170198


In [22]:
import time

camera.unobserve_all()
camera.stop()
time.sleep(1)
robot.stop()